# NaiveBayes Headlines Classification Using Spark

**This is a simple walkthrough of classification of UCI ML News Aggregator dataset using Spark**

In [50]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean, split, col, regexp_extract, when, lit, udf, regexp_replace,isnull
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

from pyspark.ml.feature import CountVectorizer, StringIndexer, RegexTokenizer, StopWordsRemover
from pyspark.sql.types import StringType, IntegerType
from pyspark.ml.classification import NaiveBayes

In [15]:
spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "512m").getOrCreate()

In [16]:
spark

In [17]:
data = spark.read.csv('data/uci-news-aggregator.csv', header='True', inferSchema='True')

In [18]:
data.show()

+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
| ID|               TITLE|                 URL|           PUBLISHER|CATEGORY|               STORY|            HOSTNAME|    TIMESTAMP|
+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
|  1|Fed official says...|http://www.latime...|   Los Angeles Times|       b|ddUyU0VZz0BRneMio...|     www.latimes.com|1394470370698|
|  2|Fed's Charles Plo...|http://www.livemi...|            Livemint|       b|ddUyU0VZz0BRneMio...|    www.livemint.com|1394470371207|
|  3|US open: Stocks f...|http://www.ifamag...|        IFA Magazine|       b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371550|
|  4|Fed risks falling...|http://www.ifamag...|        IFA Magazine|       b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371793|
|  5|Fed's Plosser: Na...|http://www.moneyn...|           Mone

In [19]:
data.count()

422937

In [20]:
title_category = data.select("TITLE","CATEGORY")

In [21]:
title_category.show()

+--------------------+--------+
|               TITLE|CATEGORY|
+--------------------+--------+
|Fed official says...|       b|
|Fed's Charles Plo...|       b|
|US open: Stocks f...|       b|
|Fed risks falling...|       b|
|Fed's Plosser: Na...|       b|
|Plosser: Fed May ...|       b|
|Fed's Plosser: Ta...|       b|
|Fed's Plosser exp...|       b|
|US jobs growth la...|       b|
|ECB unlikely to e...|       b|
|ECB unlikely to e...|       b|
|EU's half-baked b...|       b|
|Europe reaches cr...|       b|
|ECB FOCUS-Stronge...|       b|
|EU aims for deal ...|       b|
|Forex - Pound dro...|       b|
|Noyer Says Strong...|       b|
|EU Week Ahead Mar...|       b|
|ECB member Noyer ...|       b|
|Euro Anxieties Wa...|       b|
+--------------------+--------+
only showing top 20 rows



In [22]:
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [23]:
null_columns_count_list = null_value_count(title_category)

In [26]:
null_columns_count_list

[('TITLE', 389), ('CATEGORY', 516)]

In [24]:
df = spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count'])

In [25]:
df.show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                 TITLE|              389|
|              CATEGORY|              516|
+----------------------+-----------------+



In [27]:
title_category = title_category.dropna()

In [28]:
title_category.count()

422421

In [29]:
title_category.show(truncate=False)

+---------------------------------------------------------------------------+--------+
|TITLE                                                                      |CATEGORY|
+---------------------------------------------------------------------------+--------+
|Fed official says weak data caused by weather, should not slow taper       |b       |
|Fed's Charles Plosser sees high bar for change in pace of tapering         |b       |
|US open: Stocks fall after Fed official hints at accelerated tapering      |b       |
|Fed risks falling 'behind the curve', Charles Plosser says                 |b       |
|Fed's Plosser: Nasty Weather Has Curbed Job Growth                         |b       |
|Plosser: Fed May Have to Accelerate Tapering Pace                          |b       |
|Fed's Plosser: Taper pace may be too slow                                  |b       |
|Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014   |b       |
|US jobs growth last month hit by weather:F

In [30]:
title_category.select("Category").distinct().count()

265

In [31]:
title_category.groupBy("Category").count().orderBy(col("count").desc()).show(truncate=False)

+--------------------+------+
|Category            |count |
+--------------------+------+
|e                   |152127|
|b                   |115935|
|t                   |108237|
|m                   |45616 |
|Us Magazine         |31    |
|GossipCop           |20    |
|Contactmusic.com    |20    |
|Complex.com         |12    |
|CBS News            |12    |
|The Hollywood Gossip|11    |
|HipHopDX            |11    |
|We Got This Covered |10    |
|HeadlinePlanet.com  |10    |
|Gamepur             |8     |
|TooFab.com          |7     |
|Wetpaint            |7     |
|WorstPreviews.com   |7     |
|Consequence of Sound|7     |
|The Escapist        |6     |
|Reality TV World    |5     |
+--------------------+------+
only showing top 20 rows



In [32]:
title_category.groupBy("TITLE").count().orderBy(col("count").desc()).show(truncate=False)

+----------------------------------------------------------------------------------+-----+
|TITLE                                                                             |count|
+----------------------------------------------------------------------------------+-----+
|The article requested cannot be found! Please refresh your browser or go back  ...|145  |
|Business Highlights                                                               |59   |
|Posted by Parvez Jabri                                                            |59   |
|Posted by Imaduddin                                                               |53   |
|Posted by Shoaib-ur-Rehman Siddiqui                                               |52   |
|(click the phrases to see a list)                                                 |51   |
|Business Wire                                                                     |41   |
|PR Newswire                                                                       |38   |

In [36]:
title_category = title_category.withColumn("only_str", regexp_replace(col('TITLE'), '\d+', ''))

In [37]:
title_category.select("TITLE","only_str").show(truncate=False)

+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
|TITLE                                                                      |only_str                                                                   |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
|Fed official says weak data caused by weather, should not slow taper       |Fed official says weak data caused by weather, should not slow taper       |
|Fed's Charles Plosser sees high bar for change in pace of tapering         |Fed's Charles Plosser sees high bar for change in pace of tapering         |
|US open: Stocks fall after Fed official hints at accelerated tapering      |US open: Stocks fall after Fed official hints at accelerated tapering      |
|Fed risks falling 'behind the curve', Charles Plosser says                 

In [38]:
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(title_category)

In [39]:
raw_words.show()

+--------------------+--------+--------------------+--------------------+
|               TITLE|CATEGORY|            only_str|               words|
+--------------------+--------+--------------------+--------------------+
|Fed official says...|       b|Fed official says...|[fed, official, s...|
|Fed's Charles Plo...|       b|Fed's Charles Plo...|[fed, s, charles,...|
|US open: Stocks f...|       b|US open: Stocks f...|[us, open, stocks...|
|Fed risks falling...|       b|Fed risks falling...|[fed, risks, fall...|
|Fed's Plosser: Na...|       b|Fed's Plosser: Na...|[fed, s, plosser,...|
|Plosser: Fed May ...|       b|Plosser: Fed May ...|[plosser, fed, ma...|
|Fed's Plosser: Ta...|       b|Fed's Plosser: Ta...|[fed, s, plosser,...|
|Fed's Plosser exp...|       b|Fed's Plosser exp...|[fed, s, plosser,...|
|US jobs growth la...|       b|US jobs growth la...|[us, jobs, growth...|
|ECB unlikely to e...|       b|ECB unlikely to e...|[ecb, unlikely, t...|
|ECB unlikely to e...|       b|ECB unl

In [40]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)

In [41]:
words_df.select("words","filtered").show()

+--------------------+--------------------+
|               words|            filtered|
+--------------------+--------------------+
|[fed, official, s...|[fed, official, s...|
|[fed, s, charles,...|[fed, charles, pl...|
|[us, open, stocks...|[us, open, stocks...|
|[fed, risks, fall...|[fed, risks, fall...|
|[fed, s, plosser,...|[fed, plosser, na...|
|[plosser, fed, ma...|[plosser, fed, ma...|
|[fed, s, plosser,...|[fed, plosser, ta...|
|[fed, s, plosser,...|[fed, plosser, ex...|
|[us, jobs, growth...|[us, jobs, growth...|
|[ecb, unlikely, t...|[ecb, unlikely, e...|
|[ecb, unlikely, t...|[ecb, unlikely, e...|
|[eu, s, half, bak...|[eu, half, baked,...|
|[europe, reaches,...|[europe, reaches,...|
|[ecb, focus, stro...|[ecb, focus, stro...|
|[eu, aims, for, d...|[eu, aims, deal, ...|
|[forex, pound, dr...|[forex, pound, dr...|
|[noyer, says, str...|[noyer, says, str...|
|[eu, week, ahead,...|[eu, week, ahead,...|
|[ecb, member, noy...|[ecb, member, noy...|
|[euro, anxieties,...|[euro, anx

In [42]:
indexer = StringIndexer(inputCol="CATEGORY", outputCol="categoryIndex")
feature_data = indexer.fit(words_df).transform(words_df)

In [43]:
feature_data.select("CATEGORY","categoryIndex").show()

+--------+-------------+
|CATEGORY|categoryIndex|
+--------+-------------+
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
|       b|          1.0|
+--------+-------------+
only showing top 20 rows



In [44]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(feature_data)
countVectorizer_feateures = model.transform(feature_data)

In [45]:
countVectorizer_feateures.show()

+--------------------+--------+--------------------+--------------------+--------------------+-------------+--------------------+
|               TITLE|CATEGORY|            only_str|               words|            filtered|categoryIndex|            features|
+--------------------+--------+--------------------+--------------------+--------------------+-------------+--------------------+
|Fed official says...|       b|Fed official says...|[fed, official, s...|[fed, official, s...|          1.0|(49043,[5,42,112,...|
|Fed's Charles Plo...|       b|Fed's Charles Plo...|[fed, s, charles,...|[fed, charles, pl...|          1.0|(49043,[58,84,112...|
|US open: Stocks f...|       b|US open: Stocks f...|[us, open, stocks...|[us, open, stocks...|          1.0|(49043,[1,27,112,...|
|Fed risks falling...|       b|Fed risks falling...|[fed, risks, fall...|[fed, risks, fall...|          1.0|(49043,[5,112,578...|
|Fed's Plosser: Na...|       b|Fed's Plosser: Na...|[fed, s, plosser,...|[fed, plosser, na

In [46]:
(trainingData, testData) = countVectorizer_feateures.randomSplit([0.8, 0.2],seed = 11)

In [47]:
nb = NaiveBayes(modelType="multinomial",labelCol="categoryIndex", featuresCol="features")
nbModel = nb.fit(trainingData)
nb_predictions = nbModel.transform(testData)

In [48]:
nb_predictions.select("prediction", "categoryIndex", "features").show(5)

22/07/22 09:04:25 WARN DAGScheduler: Broadcasting large task binary with size 86.6 MiB


+----------+-------------+--------------------+
|prediction|categoryIndex|            features|
+----------+-------------+--------------------+
|       1.0|          0.0|(49043,[15,22,26,...|
|       0.0|         20.0|(49043,[74,113,39...|
|       0.0|          0.0|(49043,[21,50,51,...|
|       0.0|          0.0|(49043,[21,50,62,...|
|       0.0|          0.0|(49043,[6,21,22,5...|
+----------+-------------+--------------------+
only showing top 5 rows



In [49]:
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

22/07/22 09:04:49 WARN DAGScheduler: Broadcasting large task binary with size 86.7 MiB


Accuracy of NaiveBayes is = 0.926257
Test Error of NaiveBayes = 0.0737432 
